# Explainer Notebook for Final Project
### Group 72 
#### 02806 Social Data Analysis and Visualization

s193602 | s203520 | s203581

## 1. Motivation

Our data is a curated selection from IMDb, which offers publicly available metadata on film and TV shows. Specifically, we worked with structured information on titles, genres, release years, and user ratings to explore how the film industry has evolved over time with a focus on genre distribution. We chose this dataset as all members of our group enjoy film and TV, though we are split on which genres are most enjoyable to watch. This fact led to a discussion on the evolution of genre. This particular dataset comes from the most widely used and recognized film/TV database, and contains the exact information on genre needed to perform this analysis. 

Our goal was to create a fun and interactive experience that highlights how film genres have shifted according to time and geography. We wanted users to be able to explore not just how many movies were made in a given year, but what kinds of stories were being told, and how that has changed across decades, locations and seasons. We aimed to make complex temporal and categorical data look accessible and interesting. 



## 2. Basic stats

IMDb provides datasets with movies and a variety of metadata. The data is stored in different files, depending on the content. We combined three of the datasets:

**title.basics.tsv.gz (main metadata)**

**title.ratings.tsv.gz (user ratings)**

**title.akas.tsv.gz (alternative names/regions)**  

### Combined Dataset Overview

After merging the three IMDb datasets, we obtained a structured dataframe with **714,015 rows**. The table below summarizes the basic statistics for each column:

As IMDb doesn't provide explicit production country fields in title.basics, we extracted the region field from the akas dataset as a proxy, aggregating all regions associated with each title into a |-separated string.

| Statistic       | tconst    | titleType | primaryTitle | originalTitle | isAdult | startYear | endYear | runtimeMinutes | genres | averageRating | numVotes | production_country |
|----------------|-----------|-----------|---------------|----------------|---------|------------|----------|------------------|--------|----------------|-----------|---------------------|
| **count**       | 714,015   | 714,015   | 714,013       | 714,013        | 714,015 | 608,940    | 0        | 450,303          | 637,026| 329,613        | 329,613   | 699,897             |
| **unique**      | 714,015   | 1         | 613,714       | 627,750        | 2       | 139        | 0        | 510              | 1,494  | 91             | 19,061    | 118,177             |
| **top**         | tt0000009 | movie     | Broken        | Broken         | 0       | 2022       | NaN      | 90               | Drama  | 6.2            | 9         | US                  |
| **freq**        | 1         | 714,015   | 63            | 62             | 704,832 | 20,755     | NaN      | 29,021           | 131,785| 11,004         | 7,518     | 136,485             |

We are only interested in the following columns:

'tconst', 'primaryTitle', 'originalTitle', 'startYear',
    'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'production_country'

So we filter using that, and change the names. This gives us the final dataset with the following columns.


The dataset contains the following variables:  
- **title_id**: Unique identifier for each title.  
- **title**: Display title of the movie.  
- **original_title**: Original title of the movie.  
- **release_year**: Year the movie was released.  
- **runtime_minutes**: Duration of the movie in minutes.  
- **genre**: Genre(s) of the movie.  
- **imdb_rating**: IMDb user rating for the movie.  
- **vote_count**: Number of votes received for the IMDb rating.  
- **production_country**: Countries where the movie was produced.  


The dataset has quite a lot of missing values. This means, that not all movies can be used for every plot. Depending on the plot the data is filtered to not contain any missing values.

### Final data

| Column | Data Type | Non-Missing | Missing | Unique Values | Example |
|--------|------------|--------------|---------|----------------|---------|
| title_id | object | 714015 | 0 | 714015 | tt0000009 |
| title | object | 714013 | 2 | 613714 | Miss Jerry |
| original_title | object | 714013 | 2 | 627750 | Miss Jerry |
| release_year | Int64 | 608940 | 105075 | 139 | 1894 |
| runtime_minutes | Int64 | 450303 | 263712 | 510 | 45 |
| genre | object | 637026 | 76989 | 1494 | Romance |
| imdb_rating | Int64 | 329613 | 384402 | 91 | 5.4 |
| vote_count | Int64 | 329613 | 384402 | 19061 | 223 |
| production_country | object | 699897 | 14118 | 118177 | AU|DE|HU|US |

## 3. Data Analysis

The data in the IMDb dataset was fairly clean to begin with. That made the data handling a lot easier. For many of the plots, the only datahandling needed was simply to count or average by year or genre or country.

Each movie can have multiple genres, and the dataset contained a total of 28 unique genres across all movies:
- ['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']

We decided to focus on the 10 most popular genres of the dataset for plots. So the dataset was further filtered to only contain movies which contain one of the following focus genres

##### Focus genres
- ['Drama', 'Comedy', 'Documentary', 'Romance', 'Action', 'Crime', 'Thriller', 'Horror', 'Adventure', 'Mystery']


### Plot 1: Area chart

1. Selected relevant columns: `title`, `release_year`, and `genre`, and dropped rows with missing values.
2. Split multiple genres into separate rows using `.explode()` after splitting the genre strings.
3. Trimmed whitespace from genre names.
4. Filtered to include only genres in `FOCUS_GENRES`.
5. Limited data to movies released between **1930 and 2023**.
6. Counted the number of unique movie titles per year to avoid duplicate counts across multiple genres.
7. Computed yearly counts of movies per genre using `.groupby()` and `.unstack()`.
8. Reindexed genre columns to match `FOCUS_GENRES` and filled missing values with 0.
9. Calculated the percentage share of each genre per year among the `FOCUS_GENRES`.

### Plot 2: IMDb rating and runtime barchart

1. Loaded the cleaned movie dataset from a `.parquet` file.
2. Defined a function to extract the first matching genre from the `genre` field based on a predefined `MOVIE_GENRES` list.
3. Created a new column `primary_genre` with the first valid genre per movie.
4. Filtered out rows with missing or non-matching `primary_genre` values.
5. Converted `imdb_rating` and `runtime_minutes` columns to numeric, coercing invalid entries to `NaN`.
6. Grouped data by `primary_genre` and computed:
   - Average IMDb rating
   - Average runtime (in minutes)
   - Movie count per genre
7. Dropped any remaining rows with missing values from the summary table.

### Plot 3: Map plot
1. Loaded the movie dataset from a `.parquet` file.
2. Split the `production_country` field (pipe-separated string) into lists and exploded into separate rows.
3. Converted 2-letter country codes to 3-letter ISO Alpha-3 codes using the `pycountry` package.
4. Filtered out rows where country conversion failed (could not be mapped to a valid ISO Alpha-3 code).
5. Mapped 3-letter country codes to full country names for improved hover text.
6. Split the `genre` field by comma and exploded the list to associate each movie-country row with individual genres.
7. Stripped whitespace from genre names to ensure clean and consistent values.

### Plot 4: Polar plot

For the polar plot, we needed exact date releases for the movies. However, the IMDb dataset only contains release year. To acquire the precise release date, the TMDB database API was used. Using a personal API key, precise dates could be requested for each movie ID. But the API limits meant it took a lot of time, estimated 30 hours for 100k movies. Using parallelized requests, we gathered 50k exact movie titles for the polar plot. We sorted the data before, to only get movies after 1980s. This would allow the conclusions of the plot, to fit better with modern traditions and holidays etc. 

1. Loaded movie dataset and filtered rows with successful date fetch status.
2. Converted `release_date_full` to datetime and removed invalid dates (e.g., January 1st). This was removed because an unusual amount of movies seemed to have the date as 1. january, we think this is an error from the TMDB dataset api.
3. Extracted calendar components:
   - ISO week (`release_week`)
   - Calendar month (`release_month`)
   - Day of year (`release_dayofyear`)
4. Split and exploded the `genre` column to allow per-genre analysis.
5. Filtered to include only genres in `MOVIE_GENRES`.
6. **Weekly Data**:
   - Counted movies per genre per ISO week.
   - Merged with a complete week-genre template to fill missing combinations.
   - Calculated angular positions for polar coordinates (`theta_degrees`).
   - Generated total weekly movie counts and formatted week labels for hover text.
7. **Monthly Data**:
   - Counted movies per genre per month.
   - Merged with a month-genre template and computed `theta_degrees`.
   - Created hover labels using month abbreviations.
8. **Daily Data**:
   - Counted movies per genre per day of year (excluding day 1).
   - Used a full day-of-year template (days 2–365) for consistency.
   - Mapped day numbers to readable labels using a base year for formatting.
   - Calculated angular positions and joined all relevant info for hover interaction.


### Top 10 Movie be Genre Overview

1. Data was filtered, to contain the top 15 movies for every genre with at least 30k votes
2. Using the TMDB API, the posters for each of the movies was requested. 
3. The movies with no poster on the TMDB database, were filtered away, to reveal the top 10 movies WITH psoters.
4. The webpage overview, was made with a lot of assistance from Gemini 2.5 Pro.


## 4. Genre

The genre of our data story is **magazine style**, as it tells a coherent, linear narrative. We felt this format fit well with the temporal nature of our theme and allowed users to scroll through our analysis in a simple, intuitive way. The idea was, that this format would allow us to combine storytelling with interactive plots, offering a guided explanation while also allowing the user to discover patterns in the data themselves. 

### Visual Narrative

From the *Visual Narrative* categories laid out by Segal and Heer, we apply several key techniques.

#### a. Visual Structuring

Naturally, our visual story begins with an **establishing shot** that gives users an overview of the dataset and the genres at play. We also use a **consistent visual platform** by retaining genre-specific colors throughout the analysis. Finally, due to the temporal nature of our data, we incorporate a **timebar** to show how genre popularity evolves through the years. 

#### b. Highlighting

We leave the **close-ups** to the user. Our interactive graphs allow them to focus on specific genres through hover and click. We also employ **feature distinction** through our consistent genre color scheme. **Motion** is used through hover animations, which help users engage with the plot and "zoom in" on a given year.

#### c. Transition Guidance

The **familiar objects** throughout our data story are the different genres and their associated colors. We apply **continuity editing** and **object continuity** to maintain a coherent visual flow, reinforced by the consistent visual styling. While we don’t use elaborate animated transitions, our interactive plots include zoom and selection animations that support a smooth user experience.

### Narrative Structure

#### a. Ordering

Our project follows a **linear flow**, in line with the magazine-style layout. We also include elements of a **user-directed path** through interactive components.

#### b. Interactivity

Our plots include **hover highlighting**, **filtering/selection**, and **navigation controls**, most notably in Figure 2, which allows the user to select a genre and view its global distribution. While the plots themselves don’t provide **explicit instruction**, the figure captions do, and the interactions are easily discoverable via **tacit tutorial**. Additionally, the plots display **stimulating default views**, such as total movie production and genre share, that invite exploration.

#### c. Messaging

Our visualizations are supported by clear **captions/headlines** that explain their content. Relevant plots (e.g., the polar plot) include **annotations**. Since we follow a magazine structure, each plot is accompanied by explanatory text that builds a **narrative** around the data. This structure also includes **introductory text** to frame the project, and a **summary** to wrap it up.


## 5. Visualization

To support our narrative of how movie genres reflect society, we chose four complementary and interactive visualizations that each highlight a different dimension of the data. We also added an optional webpage, that shows top movies for each genre. 

#### Annual Genre Distribution (Stacked Area Chart) and barchart distribution of movie production

The first visualization shows the relative share of each genre per year from 1930 to 2023. Over it, we added a discreet minimalistic bar chart indicating the total number of movies produced for that year to make sense of the percentages of the movies. This view gives users an immediate sense of how certain genres have grown or shrunk in popularity, as well as how the film industry has expanded overall. It also shows how movie production has exploded in the past few decades. It provides historical context and serves as a strong visual foundation for the rest of the article. 

This plot was made interactive, so the user could hover each year, and see the exact relative percentages of each genre. Without that feature, it could prove difficult to see the precise share of each movie for specific years. It also allows the user to zoom in on specific periods of interest, letting the user explore the data on his/her own. 

#### Top movie genres

We made a webpage, showing top 10 rated movies for each genre. This is an optional link, the user can click, if he/she is interested. It functions as a link between the numbers, and what the data is actually about: movies! We wanted to get the most out of the dataset, and this overview might provide useful recommendations for the user.


#### Average IMDb Rating and Average Runtime by movie genre

We have previously shown the distribution and popularity of different movie genres over time. Now we want to zoom in a bit, and see what our dataset has to show about the focus genres. The dataset contains runtime and imdb ratings, so we made a very simple sorted barchart of average runtime and imdb ratings. The user most likely already has an idea about differences between the genres, but this quantifies it.

This plot is also interactive. This allows us to save space and not have two similar barcharts (rating and runtime) next to eachother. While it is indeed a very simple chart, and could be done in a static barchart, we chose to make all plots including this one interactive to keep a visual consistency on the webpage.


#### Global Genre Specialization (Map Plot)

Next, we include an interactive map that shows genre specialization across countries. Initially we wanted an interactive map timeline plot, showing each contries favorite genre over years spanning from 1930-2023. However, almost every country on the planet's favorite genre is Drama. That made for an extremely boring monocoloured map.

Instead, we opted to show the genre specialization for each focus genre. This is calculated as each genres particular share, of all movies produced in a given country. This plot would allow the user to investigate which countries enjoy what genres. Generally, the distribution is somewhat similar across countries, but there are notable exceptions, some of which are discussed on the website. 

This plot was also made interactive, so the user can explore the map (zooming, moving, hovering) and select the genre of interest. 

#### Weekly Release Patterns (Polar Chart)

Finally, our article features a circular bar chart (polar plot) that visualizes weekly, monthly and daily movie release patterns across the focus genres. The polar plot type was chosen, due to the cyclical nature of a year, allowing us to nicely aggregate data from weeks, months and days over many years. This format reveals seasonality. For example, certain genres may be released more often in specific parts of the year. 

This plot, was also made interactive. This has several advantages in the case of this plot. 

1. While a polar chart can look nice, it can also have the disadvantage of making the y-axis labels difficult to see over the bars. An interactive plot, allows the user to quickly get counts and details, by hovering stacked bars of interest. 

2. The user can choose which genres to look at, and which time scale: weeks, months or days. For example, the user might be interested in checking which months horror movies are most frequently released, could it be October? And perhaps, an abundance of them are released on halloween? 

Together, these visualizations offer a temporal, geographic, and seasonal lens on the same dataset. Their interactivity makes them engaging and allows users to explore their own questions while following a coherent story. Because they are interactive, the plots are also highly information-dense, allowing users to reveal detailed insights on demand without overwhelming the layout visually.



## 6. Discussion

The first idea, which we explored in the video of project A, was to make a similar genre based story, but for books. Unfortunately, the data for books, was just not available to the extent we needed it for. Luckily, the IMDb dataset came to our rescue with the freely provided data from their API. This dataset, was luckily extremely clean and easy to access, making the transition very easy.

We decided to focus on 10 genres, and make overviews visualizing the IMDb data for these, with temporal, geographic and seasonal lenses. While there are 28 Genres in the dataset, we deliberately narrowed our scope to avoid overwhelming the viewer and to maintain narrative cohesion. 

Movies in this dataset often belong to multiple genres. This can make conclusions difficult to draw. For example, a lot of movies have the Drama tag, so this genre can span a variety of movie types, and the same goes for several of the other genres. This makes it hard to pinpoint what a Drama movie actually is. To address this, we "exploded" the genre field and treated each genre entry individually. While this can potentially inflate counts for unique genres and alter comparisons, we chose this approach over simplifications like using only the first-listed genre, which would have introduced other biases. Nonetheless, this method must be interpreted with caution, especially when considering overrepresented umbrella genres.

We initially planned a map to visualize the “favorite genre per country,” but unfortunately this proved to be a boring monocoloured visualization, as the favorite genre of every country, across almost all years, is drama. Instead we opted for genre specialization. This highlights where certain genres are relatively overrepresented compared to global trends. This approach provides more nuanced insight and better invites exploration. However, this meant we had to filter the dataset not only for focus genres, but also for country codes, making the dataset vastly smaller. When making this filter, we discovered that many movies are missing or inconsistently tagged with country codes. That means that some countries contain little or no data for specific genres. This gives a result with potentially a lot of error for smaller countries. In the worst case, this could be misleading, however, the user has the option to hover and see stats for that country and be critical of the data. Furthermore, the country codes in the dataset may not always contain the same meaning. Ideally, the country code reflects the country of production, and this is largely the case. However, it seems that for some movies, it can represent the possible languages of the movies, or it can represent which countries the movie were shown in the theaters. This is a strong limitation of the geographical aspect of the data, and could lead to misleading results. Especially for smaller countries with low data volume. However, given the size of the dataset, we decided to preserve transparency by letting users interact directly with the data via hover tools, allowing them to make informed judgments.

Another minor limitation of the dataset relates to the precise release dates gathered from the TDBM API. If no exact date is available, this API will default to the first of the month/year. This was partly accounted for by removing all movies with a release date on January 1st, as this was drastically increased. However, the first date of the remaining months may have inflated release date counts. We chose not to remove films released on the first of Feb-Dec as that would affect the monthly aggregation, and the change in these months was not as significant. 

The plots are all interactive. This enhances engagement and flexibility, although it comes with a trade-off: slightly longer load times. We prioritized interactivity as we believe it better supports open-ended investigation, which is especially important when working with datasets that have known uncertainties or potential for misinterpretation.

Overall, our web story balances narrative flow with exploratory freedom. It acknowledges the imperfections of the data and the complexity of classification while still offering users the tools and context needed to draw meaningful insights.


## 7. Contributions

|                             | Main Resposible | s193602 | s203520 | s203581 |
|-----------------------------|-----------------|---------|---------|---------|
| Website                     | s193602         | 55%     | 22.5%   | 22.5%   |
| Introduction                | s203581         | 25%     | 25%     | 50%     |
| Film genres throughout time | s203581         | 30%     | 30%     | 40%     |
| Map plot                    | s203520         | 30%     | 40%     | 30%     |
| Polar plot                  | s203520         | 22.5%     | 55%     | 22.5%     |
| Notebook Explainer          | s193602         | 40%     | 30%     | 30%     |